In [42]:
import numpy as np
import tracemalloc
import os
import linecache
import time
import pandas as pd

In [36]:
# generate file with 100000 lines
with open('data/new_long_file.csv', 'w+') as new_file:
    for i in range(0,100000):
        new_file.write(str(np.random.randint(0, 100)))
        new_file.write('\n')

In [37]:
def display_top(snapshot, key_type='lineno', limit=3):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        # replace "/path/to/module/file.py" with "module/file.py"
        filename = os.sep.join(frame.filename.split(os.sep)[-2:])
        print("#%s: %s:%s: %.1f KiB"
              % (index, filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

In [49]:
tracemalloc.start()
start = time.clock()
with open('data/new_long_file.csv', 'r') as f:
    numbers = []
    for line in f:
        numbers.append(int(line.strip()))

print(np.mean(numbers))

snapshot = tracemalloc.take_snapshot()
display_top(snapshot)
stop = time.clock()

print('finished in', stop-start, 'seconds')

49.53308
Top 3 lines
#1: <frozen importlib._bootstrap_external>:487: 11156.3 KiB
#2: core/internals.py:4821: 1562.7 KiB
    stacked = np.empty(shape, dtype=dtype)
#3: python3.6/linecache.py:137: 1059.4 KiB
    lines = fp.readlines()
11166 other: 8001.4 KiB
Total allocated size: 21779.8 KiB
finished in 2.464156 seconds


In [46]:
tracemalloc.start()
start = time.clock()
current_mean = pd.read_csv('data/new_long_file.csv', header=None).mean()
print(current_mean)

snapshot = tracemalloc.take_snapshot()
display_top(snapshot)
stop = time.clock()

print('finished in', stop-start, 'seconds')

0    49.53308
dtype: float64
Top 3 lines
#1: <frozen importlib._bootstrap_external>:487: 10131.0 KiB
#2: core/internals.py:4821: 1562.7 KiB
    stacked = np.empty(shape, dtype=dtype)
#3: python3.6/linecache.py:137: 611.4 KiB
    lines = fp.readlines()
9486 other: 5952.3 KiB
Total allocated size: 18257.5 KiB
finished in 1.8444669999999999 seconds


In [41]:
tracemalloc.start()
start = time.clock()
with open('data/new_long_file.csv', 'r') as f:
    counter = 0
    current_mean = 0
    for line in f:
        current_mean = (current_mean*counter+int(line.strip()))/(counter+1)
        counter += 1
print(current_mean)

snapshot = tracemalloc.take_snapshot()
display_top(snapshot)
stop = time.clock()

print('finished in', stop-start, 'seconds')

49.53307999999989
Top 3 lines
#1: python3.6/linecache.py:137: 32.5 KiB
    lines = fp.readlines()
#2: core/compilerop.py:99: 7.0 KiB
    return compile(source, filename, symbol, self.flags | PyCF_ONLY_AST, 1)
#3: python3.6/codeop.py:133: 3.0 KiB
    codeob = compile(source, filename, symbol, self.flags, 1)
105 other: 31.4 KiB
Total allocated size: 73.9 KiB
finished in 0.22972899999999985 seconds


# Zadanie

Zmierzcie czas i pamięć analogicznie trzech podejść do znalezienia wartości maksymalnej i minimalnej z pliku.
* Napiszcie skrypty wykonujące trzy podejścia w osobnych plikach.
* W czwartym pliku wywołajcie te trzy funkcje i zmierzchnie ich czas i zajętą pamięć.